In [1]:
from tuning.utils.utils import chat_template_func
from vllm import LLM, SamplingParams
from tuning.config import MODELS_DIR
from tuning.inference.config_inference import VLLMSamplingParamsConfig
from tuning.data.test_dataset import get_ifeval_test_dataset
import os
import wandb
wandb.disabled = True
os.environ['WANDB_DISABLED'] = 'true'

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 02-06 12:15:35 [__init__.py:244] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!


/home/shougan/projects/aip-fredashi/shougan/balance-budget/venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/shougan/projects/aip-fredashi/shougan/balance-budget/venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annota

/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning


In [2]:
model_name = "llama3-1B"
model_path = os.path.join(MODELS_DIR, model_name)
llm = LLM(model=model_path)

INFO 02-06 12:15:56 [config.py:841] This model supports multiple tasks: {'classify', 'generate', 'reward', 'embed'}. Defaulting to 'generate'.


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 02-06 12:15:56 [config.py:3368] Downcasting torch.float32 to torch.bfloat16.
INFO 02-06 12:15:56 [config.py:1472] Using max model len 131072
INFO 02-06 12:15:59 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=16384.
WARNING 02-06 12:15:59 [__init__.py:2662] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized
INFO 02-06 12:16:06 [__init__.py:244] Automatically detected platform cuda.
INFO 02-06 12:16:08 [core.py:526] Waiting for init message from front-end.
INFO 02-06 12:16:08 [core.py:69] Initializing a V1 LLM engine (v0.9.2) with config: model='/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/models/llama3-1B', speculative_config=None, tokenizer='/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/models/llama3-1B', skip_tokenizer_init=Fals

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.92s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.92s/it]



INFO 02-06 12:16:11 [default_loader.py:272] Loading weights took 2.00 seconds
INFO 02-06 12:16:11 [gpu_model_runner.py:1801] Model loading took 2.3205 GiB and 2.460022 seconds
INFO 02-06 12:16:19 [backends.py:508] Using cache directory: /home/shougan/.cache/vllm/torch_compile_cache/3019068c4d/rank_0_0/backbone for vLLM's torch.compile
INFO 02-06 12:16:19 [backends.py:519] Dynamo bytecode transform time: 7.13 s
INFO 02-06 12:16:22 [backends.py:155] Directly load the compiled graph(s) for shape None from the cache, took 2.659 s
INFO 02-06 12:16:22 [monitor.py:34] torch.compile takes 7.13 s in total
INFO 02-06 12:16:23 [gpu_worker.py:232] Available KV cache memory: 64.03 GiB
INFO 02-06 12:16:23 [kv_cache_utils.py:716] GPU KV cache size: 2,097,984 tokens
INFO 02-06 12:16:23 [kv_cache_utils.py:720] Maximum concurrency for 131,072 tokens per request: 16.01x


Capturing CUDA graph shapes: 100%|██████████| 67/67 [00:17<00:00,  3.93it/s]


INFO 02-06 12:16:41 [gpu_model_runner.py:2326] Graph capturing finished in 17 secs, took 0.43 GiB
INFO 02-06 12:16:41 [core.py:172] init engine (profile, create kv cache, warmup model) took 29.56 seconds


In [3]:
config = VLLMSamplingParamsConfig()
config.n = 1
sampling_params = SamplingParams(**config.model_dump())
print(sampling_params)

SamplingParams(n=1, presence_penalty=0.0, frequency_penalty=0.0, repetition_penalty=1.0, temperature=0.5, top_p=0.9, top_k=150, min_p=0.0, seed=None, stop=['<|im_end|>', '<|end_of_text|>'], stop_token_ids=[], bad_words=[], include_stop_str_in_output=False, ignore_eos=False, max_tokens=4096, min_tokens=0, logprobs=None, prompt_logprobs=None, skip_special_tokens=True, spaces_between_special_tokens=True, truncate_prompt_tokens=None, guided_decoding=None, extra_args=None)


In [ ]:
tokenizer = llm.get_tokenizer()
tokenizer = chat_template_func(tokenizer)   
chat_template = tokenizer.chat_template
# Convert the whole list at once
ids_to_check = [128000, 27, 91, 318, 5011, 91, 29, 9125, 198, 2675, 527, 264, 11190, 18328, 889, 374, 459, 6335, 520, 30438, 311, 52032, 555, 15884, 2768, 279, 2728, 11470, 128001, 198, 27, 91, 318, 5011, 91, 29, 882, 198, 8144, 264, 220, 3101, 10, 3492, 12399, 315, 279, 59318, 2199, 330, 2485, 1129, 268, 34466, 2726, 26583, 19945, 352, 12669, 62, 23440, 21025, 2568, 3659, 1159, 4664, 14559, 3343, 3234, 539, 1005, 904, 77702, 323, 11415, 520, 3325, 220, 18, 14491, 430, 706, 15671, 304, 51594, 3645, 11, 369, 3187, 353, 36298, 291, 3857, 961, 220, 16, 12594, 353, 36298, 291, 3857, 961, 220, 17, 12594, 353, 36298, 291, 3857, 961, 220, 18, 20517, 128001, 198, 27, 91, 318, 5011, 91, 29, 78191, 198]
tokens = tokenizer.convert_ids_to_tokens(ids_to_check)
print(tokens)

['<|begin_of_text|>', '<|im_end|>', '<', '|', 'im', '_start', '|', '>', 'system', 'Ċ', 'You', 'Ġare', 'Ġa', 'Ġhelpful', 'Ġassistant', 'Ġwho', 'Ġis', 'Ġan', 'Ġexpert', 'Ġat', 'Ġresponding', 'Ġto', 'Ġprompts', 'Ġby', 'Ġcarefully', 'Ġfollowing', 'Ġthe', 'Ġgiven', 'Ġinstructions', '<|im_end|>', 'Ċ', '<', '|', 'im', '_start', '|', '>', 'user', 'Ċ', 'Write', 'Ġa', 'Ġ', '300', '+', 'Ġword', 'Ġsummary', 'Ġof', 'Ġthe', 'Ġwikipedia', 'Ġpage', 'Ġ"', 'https', '://', 'en', '.wikipedia', '.org', '/wiki', '/R', 'ay', 'mond', '_', 'III', ',_', 'Count', '_of', '_T', 'rip', 'oli', '".', 'ĠDo', 'Ġnot', 'Ġuse', 'Ġany', 'Ġcommas', 'Ġand', 'Ġhighlight', 'Ġat', 'Ġleast', 'Ġ', '3', 'Ġsections', 'Ġthat', 'Ġhas', 'Ġtitles', 'Ġin', 'Ġmarkdown', 'Ġformat', ',', 'Ġfor', 'Ġexample', 'Ġ*', 'highlight', 'ed', 'Ġsection', 'Ġpart', 'Ġ', '1', '*,', 'Ġ*', 'highlight', 'ed', 'Ġsection', 'Ġpart', 'Ġ', '2', '*,', 'Ġ*', 'highlight', 'ed', 'Ġsection', 'Ġpart', 'Ġ', '3', '*.', '<|im_end|>', 'Ċ', '<', '|', 'im', '_start', '|', 

In [14]:
special_tokens = ["<|begin_of_text|>", "<|start_header_id|>", "<|eot_id|>", "<|end_of_text|>", "<|im_end|>", '<|im_start|>']
for token in special_tokens:
    print(f"{token}: {tokenizer.convert_tokens_to_ids(token)}")

<|begin_of_text|>: 128000
<|start_header_id|>: 128006
<|eot_id|>: 128009
<|end_of_text|>: 128256
<|im_end|>: 128001
<|im_start|>: None


In [5]:
dataset = get_ifeval_test_dataset()

In [14]:
outputs = llm.chat(
    dataset["messages"][:5],  # Just run on first 10 samples for demo
    sampling_params, 
    chat_template=chat_template,
)

Adding requests:   0%|          | 0/5 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/21 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

KeyboardInterrupt: 